In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from random import choice
pd.options.mode.chained_assignment = None 
import pickle

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Coding/Python/'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
# from lonelyboy.geospatial import group_patterns as gsgp


# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')
get_ipython().magic('matplotlib qt')
# get_ipython().magic('matplotlib inline')

style.use('ggplot')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [3]:
# PLT_FIG_WIDTH = 13.487
PLT_FIG_WIDTH = 3.787
PLT_FIG_HEIGHT = PLT_FIG_WIDTH / 1.618

In [4]:
(PLT_FIG_WIDTH, PLT_FIG_HEIGHT)

(3.787, 2.3405438813349813)

## Vessel-Static Data Statistics

In [5]:
vessel_static = pd.read_csv('./data/csv/nari_static.csv')
vessel_cat_list = pd.read_csv('./data/csv/Ship Types List.csv')
vessel_det_cat_list = pd.read_csv('./data/csv/Ship Types Detailed List.csv')

C:\Users\This PC\.conda\envs\gis\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
print (f'Before: {len(vessel_static)}')
vessel_static = vessel_static.dropna(subset=['shiptype'])
vessel_static = vessel_static.drop_duplicates(subset=['sourcemmsi', 'shiptype'], keep='first')
print (f'After: {len(vessel_static)}')

Before: 1078617
After: 5011


In [7]:
vessel_static.head(10)

,sourcemmsi,imonumber,callsign,shipname,shiptype,tobow,tostern,tostarboard,toport,eta,draught,destination,mothershipmmsi,t
0,304091000,9509255.0,V2GU5,HC JETTE-MARIT,70.0,130.0,30.0,18.0,6.0,04-09 20:00,10.1,BREST,NaN,1443650423
1,228037600,0.0,FIHX,AEROUANT BREIZH,30.0,6.0,9.0,5.0,2.0,00-00 24:60,0.0,,NaN,1443650457
2,228064900,8304816.0,FITO,VN SAPEUR,51.0,21.0,54.0,10.0,6.0,29-09 12:00,5.9,RADE DE BREST,NaN,1443650471
3,227705102,262144.0,FGD5860,BINDY,60.0,9.0,26.0,5.0,4.0,00-00 24:60,0.0,,NaN,1443650474
4,227415000,0.0,FHAF,F/V JEREMI SIMON,90.0,11.0,9.0,3.0,3.0,00-00 24:60,0.0,,NaN,1443650479
5,226263000,9308687.0,FZTC,ABEILLE BOURBON,51.0,36.0,44.0,8.0,8.0,31-12 00:00,6.0,SALVAGE TUG,NaN,1443650516
6,245257000,9148142.0,PFHD,EEMS CARRIER,70.0,75.0,9.0,3.0,8.0,30-09 17:00,2.7,BREST,NaN,1443650535
7,227002330,0.0,FO4922,ANDRE COLIN,60.0,15.0,20.0,2.0,5.0,25-09 10:30,1.8,BREST,NaN,1443650591
8,227003050,0.0,FV5533,ENEZ EUSSA 3,60.0,25.0,20.0,4.0,5.0,29-09 19:50,2.8,BREST,NaN,1443650611
9,227443000,0.0,FITP,F/V PROMETHEE,30.0,11.0,9.0,3.0,3.0,00-00 24:60,0.0,,NaN,1443650629


In [8]:
len(vessel_static.sourcemmsi.unique())

4767

In [9]:
for mmsi, mmsi_static in vessel_static.groupby('sourcemmsi'):
    ves_types = [vessel_cat_list.loc[(shiptypeID >= vessel_cat_list.shiptype_min) & (shiptypeID <= vessel_cat_list.shiptype_max)][['type_name', 'ais_type_summary']]['type_name'].values.tolist() for shiptypeID in mmsi_static.shiptype.unique()]
    if (len(ves_types) > 1):
        print (f'Vessel: {mmsi}\t Shiptype(s): {ves_types}')

Vessel: 209672000	 Shiptype(s): [['Tanker - Hazard B'], ['Tanker - Hazard C (Minor)'], ['Tanker']]
Vessel: 209840000	 Shiptype(s): [['Cargo - Hazard A (Major)'], ['Cargo - Hazard D (Recognizable)'], ['Cargo - Hazard B'], ['Cargo - Hazard C (Minor)']]
Vessel: 211135000	 Shiptype(s): [['Tanker'], ['Tanker']]
Vessel: 211379750	 Shiptype(s): [['Tanker'], ['Tanker']]
Vessel: 211409100	 Shiptype(s): [['Tanker'], ['Tanker']]
Vessel: 211881000	 Shiptype(s): [['Tanker'], ['Tanker']]
Vessel: 215194000	 Shiptype(s): [['Tanker - Hazard B'], ['Tanker']]
Vessel: 215234000	 Shiptype(s): [['Tanker - Hazard A (Major)'], ['Tanker']]
Vessel: 215477000	 Shiptype(s): [['Tanker'], []]
Vessel: 215581000	 Shiptype(s): [['Tanker - Hazard B'], ['Tanker']]
Vessel: 215760000	 Shiptype(s): [['Tanker - Hazard D (Recognizable)'], ['Tanker']]
Vessel: 218108000	 Shiptype(s): [['Cargo'], ['Cargo']]
Vessel: 218551000	 Shiptype(s): [['Tanker'], ['Tanker']]
Vessel: 219116000	 Shiptype(s): [['Cargo'], ['Cargo - Hazard A (M

Vessel: 256391000	 Shiptype(s): [['Tanker - Hazard B'], ['Tanker - Hazard A (Major)']]
Vessel: 256483000	 Shiptype(s): [['Tanker - Hazard B'], ['Tanker']]
Vessel: 256494000	 Shiptype(s): [['Tanker'], ['Tanker - Hazard D (Recognizable)']]
Vessel: 256714000	 Shiptype(s): [['Tanker - Hazard D (Recognizable)'], ['Tanker - Hazard B'], ['Tanker - Hazard A (Major)']]
Vessel: 256755000	 Shiptype(s): [['Tanker'], ['Tanker']]
Vessel: 256838000	 Shiptype(s): [['Tanker'], ['Tanker']]
Vessel: 257374000	 Shiptype(s): [['Tanker - Hazard B'], ['Tanker']]
Vessel: 257532000	 Shiptype(s): [['Cargo'], ['Cargo - Hazard D (Recognizable)'], ['Cargo - Hazard A (Major)']]
Vessel: 258316000	 Shiptype(s): [['Cargo - Hazard A (Major)'], ['Cargo']]
Vessel: 258682000	 Shiptype(s): [['Tanker'], ['Tanker']]
Vessel: 259004000	 Shiptype(s): [['Tanker - Hazard B'], ['Tanker - Hazard A (Major)']]
Vessel: 259985000	 Shiptype(s): [['Tanker'], ['Tanker - Hazard A (Major)']]
Vessel: 266273000	 Shiptype(s): [['Tanker'], ['Tan

In [10]:
vessel_cat_list.head()

,id_shiptype,shiptype_min,shiptype_max,type_name,ais_type_summary
0,1,10,19,Reserved,Unspecified
1,2,20,28,Wing In Grnd,Wing in Grnd
2,3,29,29,SAR Aircraft,Search and Rescue
3,4,30,30,Fishing,Fishing
4,5,31,31,Tug,Tug


In [11]:
vessel_det_cat_list.head()

,id_detailedtype,detailed_type,id_shiptype
0,1,Wing In Ground Effect Vessel,2
1,2,Trawler,4
2,3,Fishery Protection/Research,4
3,4,Fish Carrier,4
4,5,Fish Factory,4


In [12]:
vessel_categories = vessel_cat_list.merge(vessel_det_cat_list, on='id_shiptype', how='inner')
vessel_categories

,id_shiptype,shiptype_min,shiptype_max,type_name,ais_type_summary,id_detailedtype,detailed_type
0,2,20,28,Wing In Grnd,Wing in Grnd,1,Wing In Ground Effect Vessel
1,4,30,30,Fishing,Fishing,2,Trawler
2,4,30,30,Fishing,Fishing,3,Fishery Protection/Research
3,4,30,30,Fishing,Fishing,4,Fish Carrier
4,4,30,30,Fishing,Fishing,5,Fish Factory
5,4,30,30,Fishing,Fishing,6,Factory Trawler
6,4,30,30,Fishing,Fishing,7,Fish Storage Barge
7,4,30,30,Fishing,Fishing,8,Fishery Research Vessel
8,4,30,30,Fishing,Fishing,9,Fishery Patrol Vessel
9,4,30,30,Fishing,Fishing,10,Fishery Support Vessel


In [13]:
vessel_categories.type_name.unique()

array(['Wing In Grnd', 'Fishing', 'Tug', 'Dredger', 'Dive Vessel',
       'Military Ops', 'Sailing Vessel', 'Pleasure Craft',
       'High-Speed Craft', 'SAR', 'Port Tender', 'Anti-Pollution',
       'Law Enforce', 'Medical Trans', 'Special Craft', 'Passenger',
       'Cargo', 'Tanker', 'Tanker - Hazard D (Recognizable)'],
      dtype=object)

In [14]:
tmp = vessel_static.shiptype.value_counts().to_frame().reset_index()
tmp.columns = ['shiptype', '#records']

tmp['type_name'] = np.nan
tmp['ais_type_summary'] = np.nan

for row in tmp.itertuples(index=True):
    ves_type = vessel_cat_list.loc[(row.shiptype >= vessel_cat_list.shiptype_min) & (row.shiptype <= vessel_cat_list.shiptype_max)][['type_name', 'ais_type_summary']]
    if (len(ves_type) > 0):
        tmp.loc[row.Index, 'type_name'] = ves_type.type_name.values[0]
        tmp.loc[row.Index, 'ais_type_summary'] = ves_type.ais_type_summary.values[0]

tmp.fillna('Unknown', inplace=True)

In [15]:
tmp

,shiptype,#records,type_name,ais_type_summary
0,70.0,2085,Cargo,Cargo
1,80.0,543,Tanker,Tanker
2,79.0,394,Cargo,Cargo
3,30.0,357,Fishing,Fishing
4,71.0,293,Cargo - Hazard A (Major),Cargo
5,89.0,266,Tanker,Tanker
6,36.0,222,Sailing Vessel,Sailing Vessel
7,82.0,123,Tanker - Hazard B,Tanker
8,81.0,95,Tanker - Hazard A (Major),Tanker
9,90.0,82,Other,Other


In [16]:
tmp['ais_type_summary'] = tmp['ais_type_summary'].replace(to_replace=r'[ \t$]{1}', value='\n', regex=True)

In [17]:
tmp

,shiptype,#records,type_name,ais_type_summary
0,70.0,2085,Cargo,Cargo
1,80.0,543,Tanker,Tanker
2,79.0,394,Cargo,Cargo
3,30.0,357,Fishing,Fishing
4,71.0,293,Cargo - Hazard A (Major),Cargo
5,89.0,266,Tanker,Tanker
6,36.0,222,Sailing Vessel,Sailing\nVessel
7,82.0,123,Tanker - Hazard B,Tanker
8,81.0,95,Tanker - Hazard A (Major),Tanker
9,90.0,82,Other,Other


In [18]:
# # tmp.groupby('type_name')['#records'].sum().sort_values().to_frame().reset_index().plot.barh(x='type_name', y='#records', cmap='tab20', rot=25, legend=False)
# tmp.groupby('ais_type_summary')['#records'].sum().sort_values().to_frame().reset_index().plot.bar(x='ais_type_summary', y='#records', figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), title='Vessel Type Popularity', cmap='tab20', rot=0, width=0.95, legend=False)
# # plt.ylabel('Vessel Type')
# plt.xlabel('')
# plt.ylabel('#Vessel-Types')

In [19]:
tmp.loc[tmp['ais_type_summary'] == 'Cargo']['type_name']

0                               Cargo
2                               Cargo
4            Cargo - Hazard A (Major)
13    Cargo - Hazard D (Recognizable)
17                   Cargo - Hazard B
23           Cargo - Hazard C (Minor)
26                              Cargo
28                              Cargo
30                              Cargo
41                              Cargo
Name: type_name, dtype: object

In [20]:
vessel_static_on_steroids = vessel_static
vessel_static_on_steroids['type_name'] = np.nan
vessel_static_on_steroids['ais_type_summary'] = np.nan

for row in vessel_static_on_steroids.itertuples(index=True):
    ves_type = vessel_cat_list.loc[(row.shiptype >= vessel_cat_list.shiptype_min) & (row.shiptype <= vessel_cat_list.shiptype_max)][['type_name', 'ais_type_summary']]
    if (len(ves_type) > 0):
        vessel_static_on_steroids.loc[row.Index, 'type_name'] = ves_type.type_name.values[0]
        vessel_static_on_steroids.loc[row.Index, 'ais_type_summary'] = ves_type.ais_type_summary.values[0]

vessel_static_on_steroids.fillna('Unknown', inplace=True)

In [21]:
vessel_static_on_steroids

,sourcemmsi,imonumber,callsign,shipname,shiptype,tobow,tostern,tostarboard,toport,eta,draught,destination,mothershipmmsi,t,type_name,ais_type_summary
0,304091000,9.50926e+06,V2GU5,HC JETTE-MARIT,70.0,130.0,30.0,18.0,6.0,04-09 20:00,10.1,BREST,Unknown,1443650423,Cargo,Cargo
1,228037600,0,FIHX,AEROUANT BREIZH,30.0,6.0,9.0,5.0,2.0,00-00 24:60,0,,Unknown,1443650457,Fishing,Fishing
2,228064900,8.30482e+06,FITO,VN SAPEUR,51.0,21.0,54.0,10.0,6.0,29-09 12:00,5.9,RADE DE BREST,Unknown,1443650471,SAR,Search and Rescue
3,227705102,262144,FGD5860,BINDY,60.0,9.0,26.0,5.0,4.0,00-00 24:60,0,,Unknown,1443650474,Passenger,Passenger
4,227415000,0,FHAF,F/V JEREMI SIMON,90.0,11.0,9.0,3.0,3.0,00-00 24:60,0,,Unknown,1443650479,Other,Other
5,226263000,9.30869e+06,FZTC,ABEILLE BOURBON,51.0,36.0,44.0,8.0,8.0,31-12 00:00,6,SALVAGE TUG,Unknown,1443650516,SAR,Search and Rescue
6,245257000,9.14814e+06,PFHD,EEMS CARRIER,70.0,75.0,9.0,3.0,8.0,30-09 17:00,2.7,BREST,Unknown,1443650535,Cargo,Cargo
7,227002330,0,FO4922,ANDRE COLIN,60.0,15.0,20.0,2.0,5.0,25-09 10:30,1.8,BREST,Unknown,1443650591,Passenger,Passenger
8,227003050,0,FV5533,ENEZ EUSSA 3,60.0,25.0,20.0,4.0,5.0,29-09 19:50,2.8,BREST,Unknown,1443650611,Passenger,Passenger
9,227443000,0,FITP,F/V PROMETHEE,30.0,11.0,9.0,3.0,3.0,00-00 24:60,0,,Unknown,1443650629,Fishing,Fishing


# Intersecting with the Dynamic Data

In [22]:
mean_records_per_day = pd.read_csv('./data/csv/stats/mean_records_per_day.csv')
mean_records_per_day.head()

,mmsi,records_per_day
0,923166,3.000000
1,1234500,1.000000
2,9105765,1.000000
3,37100300,1167.333333
4,205067000,18.750000


In [23]:
tmp2 = pd.merge(mean_records_per_day, vessel_static_on_steroids, left_on='mmsi', right_on='sourcemmsi', how='inner', )
tmp2.head()

,mmsi,records_per_day,sourcemmsi,imonumber,callsign,shipname,shiptype,tobow,tostern,tostarboard,toport,eta,draught,destination,mothershipmmsi,t,type_name,ais_type_summary
0,1234500,1.000000,1234500,0,,TEST,30.0,0.0,0.0,0.0,0.0,07-10 16:00,0,LORIENT,Unknown,1444895020,Fishing,Fishing
1,37100300,1167.333333,37100300,7.3124e+06,HP7085,KADOKA,52.0,0.0,29.0,8.0,0.0,10-12 14:00,4.3,HARLINGEN,Unknown,1444168711,Tug,Tug
2,205204000,1323.000000,205204000,0,ORJH,NATO WARSHIP A960,35.0,38.0,54.0,6.0,8.0,00-00 24:60,3.6,NOT AVAILABLE,Unknown,1443753239,Military Ops,Special Craft
3,205277430,105.000000,205277430,Unknown,OP2774,NEREUS,36.0,18.0,0.0,1.0,8.0,Unknown,Unknown,Unknown,3.77492e+07,1444732986,Sailing Vessel,Sailing Vessel
4,205309000,646.279070,205309000,9.17414e+06,ORMP,LANGE WAPPER,33.0,36.0,100.0,3.0,24.0,24-12 22:00,6.5,BREST,Unknown,1450975747,Dredger,Special Craft


In [24]:
tmp2 = tmp2.groupby(['type_name', 'ais_type_summary']).size().sort_values(0, ascending=True).reset_index()
tmp2.columns = ['type_name', 'ais_type_summary', '#vessels']
tmp2

,type_name,ais_type_summary,#vessels
0,Anti-Pollution,Special Craft,1
1,Local Vessel,Special Craft,1
2,Law Enforce,Special Craft,2
3,High-Speed Craft,High-Speed Craft,2
4,Dive Vessel,Special Craft,2
5,Wing In Grnd,Wing in Grnd,2
6,Pilot Vessel,Special Craft,5
7,Cargo - Hazard C (Minor),Cargo,10
8,Dredger,Special Craft,14
9,Military Ops,Special Craft,18


In [25]:
# tmp2['ais_type_summary_clean'] = tmp2['ais_type_summary'].replace(to_replace=r'[ \t$]{1}', value='\n', regex=True)
tmp2['ais_type_summary_clean'] = tmp2['ais_type_summary'].apply(lambda s: '\n'.join(s.split(' ', 1)))
tmp2

,type_name,ais_type_summary,#vessels,ais_type_summary_clean
0,Anti-Pollution,Special Craft,1,Special\nCraft
1,Local Vessel,Special Craft,1,Special\nCraft
2,Law Enforce,Special Craft,2,Special\nCraft
3,High-Speed Craft,High-Speed Craft,2,High-Speed\nCraft
4,Dive Vessel,Special Craft,2,Special\nCraft
5,Wing In Grnd,Wing in Grnd,2,Wing\nin Grnd
6,Pilot Vessel,Special Craft,5,Special\nCraft
7,Cargo - Hazard C (Minor),Cargo,10,Cargo
8,Dredger,Special Craft,14,Special\nCraft
9,Military Ops,Special Craft,18,Special\nCraft


In [33]:
tmp2.groupby('ais_type_summary_clean')['#vessels'].sum().sort_values(ascending=False).to_frame().reset_index().plot.bar(x='ais_type_summary_clean', y='#vessels', figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), cmap='tab20', fontsize=7, rot=70, width=0.95, legend=False)

# plt.ylabel('Vessel Type')
plt.suptitle(r'\textbf{Vessel Type Popularity}', fontsize=8)
plt.xlabel(r'\textbf{Vessel\ Types}', fontsize=8)
plt.ylabel(r'\textbf{\#Vessels}', fontsize=8)

plt.savefig(os.path.join('.', 'experimental_figures', 'figs_final', f'Vessel_Type_Popularity.pdf'), dpi=300, bbox_inches='tight')
# plt.close('all')

In [27]:
tmp2.groupby('ais_type_summary_clean')['#vessels'].sum().apply(np.log10).fillna(0).sort_values(ascending=False).to_frame().reset_index().plot.bar(x='ais_type_summary_clean', y='#vessels', figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), cmap='tab20', fontsize=7, rot=75, width=0.95, legend=False)

# plt.ylabel('Vessel Type')
plt.suptitle(r'\textbf{Vessel type popularity}', fontsize=8)
plt.xlabel(r'\textbf{Vessel types}', fontsize=8)
plt.ylabel(r'\textbf{\#vessels ($log_{10}$ scale)}', fontsize=8)
plt.yticks(range(4),[r'$0$', r'$10^1$', r'$10^2$', r'$10^3$'])

plt.savefig(os.path.join('..', f'Vessel_Type_Popularity_V3.pdf'), dpi=300, bbox_inches='tight')
# plt.close('all')

In [34]:
tmp2['#vessels'].sum()

3930